In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


In [2]:
power = pd.read_csv("C:/Users/Bin/Documents/Datasets/EncDec-AD dataset/power_data.txt",names=["power_demand"])


In [3]:
sub_power = pd.Series(power[490:51*84*8+490].reset_index(drop=True)["power_demand"])
sub_power = sub_power[::8].reset_index(drop=True)
sub_power = sub_power.reshape(-1, 1)
scaler = MinMaxScaler()
scaler.fit(sub_power)
sub_power = scaler.transform(sub_power) 

sub_power_ = np.array([sub_power[t][0] for t in range(sub_power.shape[0])])
sub_power_list = [sub_power_[t*84:(t+1)*84] for t in range(51)]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [4]:
anomalous_indices = [11,12,16,17,19,50]
normal_indices = list(set(range(51))-set(anomalous_indices))
dataset_normal = [sub_power_list[i] for i in normal_indices]
dataset_anomalous = [sub_power_list[j] for j in anomalous_indices]

In [5]:
label = np.array(['anomaly' for _ in range(51)])
label[normal_indices] = 'normal'
label = [[x]*84 for x in label]
label = np.array(label).ravel()

In [6]:
sub_power.shape

(4284, 1)

In [7]:
data = pd.concat((pd.DataFrame(sub_power),pd.Series(label)),axis=1)

In [8]:
#data.to_csv("C:/Users/Bin/Documents/Datasets/EncDec-AD dataset/power_data_labeled_unscaled.csv",header=None,index=None)

In [9]:
test_set = data[1008:]


# Autoencoder anomaly detection

In [10]:

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import time

## Reload power_demand autoencoder from disk

In [11]:
modelpath_root ="C:/Users/Bin/Desktop/Thesis/tmp/EncDecADModel_online_init/power_demand/Try5_2_40_84/"
modelmeta_p = modelpath_root + "LSTMAutoencoder_power_2_40_84_para.ckpt.meta"
step_num = 84
hidden_num = 40
batch_num = 2
elem_num = 1

In [12]:
test_set_list = [test_set.iloc[:3192,0].as_matrix().reshape(19,2,84,1)[a] for a in range(19)]

In [13]:
sess = tf.Session()
saver = tf.train.import_meta_graph(modelmeta_p) # load trained gragh, but without the trained parameters
saver.restore(sess,tf.train.latest_checkpoint(modelpath_root))
graph = tf.get_default_graph()

p_input = graph.get_tensor_by_name("p_input:0")
p_inputs = [tf.squeeze(t, [1]) for t in tf.split(p_input, step_num, 1)] 
#        p_is_training = tf.placeholder(tf.bool)
p_is_training = graph.get_tensor_by_name("is_training_:0")

input_= tf.transpose(tf.stack(p_inputs), [1, 0, 2])    
output_ = graph.get_tensor_by_name("decoder/output_:0")

tensor_mu = graph.get_tensor_by_name("mu:0")
tensor_sigma = graph.get_tensor_by_name("sigma:0")
tensor_threshold = graph.get_tensor_by_name("threshold:0")

loss_ = graph.get_tensor_by_name("decoder/loss:0")
train_ = graph.get_operation_by_name("cond/train_")

mu = sess.run(tensor_mu)
sigma = sess.run(tensor_sigma)
threshold = sess.run(tensor_threshold)

INFO:tensorflow:Restoring parameters from C:/Users/Bin/Desktop/Thesis/tmp/EncDecADModel_online_init/power_demand/Try5_2_40_84/LSTMAutoencoder_power_2_40_84_para.ckpt


In [14]:
input_buffer = []
output_buffer = []
for data in test_set_list:
    (input_n, output_n) = sess.run([input_, output_], {p_input: data, p_is_training: False})
    input_buffer.append(input_n)
    output_buffer.append(output_n)

In [15]:
foo = pd.Series(np.array(input_buffer).ravel())
bar = pd.Series(np.array(output_buffer).ravel())
foobar = pd.concat((foo,bar),axis=1)

foobar.to_csv("C:/Users/Bin/Desktop/Thesis/tmp/EncDecADModel_online_init/power_demand/Try5_2_40_84.csv",header=['input','output'],index=None)